**this notebook is created under the traderly challange of creating an ai model capable of detecting if a text is ai generated or humanly written**

the approach i'll be taking is to find a large dataset of both ai and human written text , then finetune the NLP model BERT on it
for more options ill be doing this using both the base and large model and comparing accuracy and speed of them just to give prespective for the Traderly team to decide what to chose

#liberaries

In [ ]:
!pip install evaluate


In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer ,DataCollatorWithPadding ,  TrainingArguments , AutoModelForSequenceClassification



#Dataset
first lets get a dataset !
after a bit of reaserch i found multiple goodones , for now lets stick with this one

In [ ]:

ds = load_dataset("andythetechnerd03/AI-human-text")
ds

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['text', 'generated'],
        num_rows: 462873
    })
    test: Dataset({
        features: ['text', 'generated'],
        num_rows: 24362
    })
})

lets see some exemples

In [ ]:
nb_exemples = 5
for i in range(nb_exemples):
  print("--------------------------------------------------------------------------")
  print(f"is ai : {ds['train'][i]['generated']==0} ")

  print(f"text  : {ds['train'][i]['text'][:200]}") # text too long to be shown so 200 car is enaugh

--------------------------------------------------------------------------
is ai : True 
text  : studies have been proven that people are starting to not drive cars as much americans are buying fewer cars and getting fewer licenses there are several advantages to not driving cars everywhere benef
--------------------------------------------------------------------------
is ai : False 
text  : i disagree with you ih twenty years the humber of cars ih use will be the same or even greater that it is today there are a humber of reasons why this is likely to be the case firstly the humber of pe
--------------------------------------------------------------------------
is ai : False 
text  : hey yall today were gonna talk about this cool idea that winston churchill had about success he said that its all about going through failure without losing your enthusiasm lime whoa right let me tell
--------------------------------------------------------------------------
is ai : True 
text  : im goin

lets split the data and create our data loaders for later use

In [ ]:
ds = ds.rename_column('generated', 'labels')


In [ ]:
train_data = ds['train']
test_data = ds['test']


#Model
now we will get the weights of the base models , prepare them for finetunning and train them on the new data

In [ ]:
# Load model directly

model_name = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)


lets tokenize the dataset!

In [ ]:
def tokenization_function(row):
  return tokenizer(row['text'] , truncation=True)


tokenized_train_data = train_data.map(tokenization_function, batched=True)
tokenized_test_data = test_data.map(tokenization_function, batched=True)
#exemple
tokenized_train_data[0]

{'text': 'studies have been proven that people are starting to not drive cars as much americans are buying fewer cars and getting fewer licenses there are several advantages to not driving cars everywhere beneficial implications for carbon emissions the environment and improves safetyvaubans streets in germany are completely car free people who do not drive cars any more and ride bikes walks or rides the tram are said to be happier this way a mother of two walks and rides her bikes everywhere with her kids and does not feel as tense and stressed as she did when she was drivingalso a huge advantage is it drastically reduces greenhouse gas emissions from tailpipes diesel fuel was banned in france since tax policies favored diesel over gasoline delivery companies complained of lost revenue but exceptions were made plugin cars hybrids and cars carrying three or more passengersin paris there were days of near record pollution so they enforced a partial driving ban to clear the air of the ci

we dont have enaugh time so imma train it only on 20 k , randomized ofc so we dont get unballanced data

In [ ]:
# 1. Find indices of each label
label_0_indices = [i for i, row in enumerate(tokenized_train_data) if row['labels'] == 0]
label_1_indices = [i for i, row in enumerate(tokenized_train_data) if row['labels'] == 1]

# 2. How many from each?
min_count = min(len(label_0_indices), len(label_1_indices), 10000)

# 3. Randomly pick
import random
sampled_0_indices = random.sample(label_0_indices, min_count)
sampled_1_indices = random.sample(label_1_indices, min_count)

# 4. Combine and shuffle
all_indices = sampled_0_indices + sampled_1_indices
random.shuffle(all_indices)

# 5. Select
small_tokenized_train_data = tokenized_train_data.select(all_indices)

# now small_tokenized_train_data is your 20k balanced dataset


lets prepare the Trainer!

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(model_name , num_labels=2)
training_args = TrainingArguments(
    num_train_epochs=5,
    fp16=True,
    per_device_train_batch_size=32,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    weight_decay=1e-4,
    max_grad_norm=0.01,
    metric_for_best_model="eval_loss",
    greater_is_better=True,
    load_best_model_at_end=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to="none"

)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=small_tokenized_train_data,
    eval_dataset=tokenized_test_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

<ipython-input-15-3c59f2d985c1>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


and now we can train!

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.170100,0.402464
2,0.049300,0.031087
3,0.033700,0.037467
4,0.003300,0.048024
5,0.000400,0.094074


TrainOutput(global_step=3125, training_loss=0.04343643413424492, metrics={'train_runtime': 3294.1141, 'train_samples_per_second': 30.357, 'train_steps_per_second': 0.949, 'total_flos': 2.6311105536e+16, 'train_loss': 0.04343643413424492, 'epoch': 5.0})

the loss seems greatt , but lets test its accuracy on unseen data

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = np.mean(predictions == labels)
    return {"accuracy": accuracy}


eval_pred = trainer.predict(tokenized_test_data)

metrics = compute_metrics((eval_pred.predictions, eval_pred.label_ids))

print(metrics)



{'accuracy': np.float64(0.940316886955094)}


now thats a great accuracy and only on 20 k rows of data , givin enaugh time we will achieve better resultts, lets save the model

In [ ]:

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_path = "my_ai_detection_model"


trainer.save_model(model_path)

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)


# Example usage after reloading
def predict_ai_generated(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
    return predicted_class

example_text = "Once upon a time, in a small village tucked between two mountains, there lived a curious young boy named Leo. Every day after his chores, Leo would wander into the woods, dreaming of finding something extraordinary.\n\nOne afternoon, as he followed a stream deeper than he ever had before, he stumbled upon a hidden cave. Inside, glowing crystals lit up the walls, and in the center sat an ancient book. The book spoke of a forgotten treasure that could bring great prosperity to the village — but only if found with a pure heart.\n\nDetermined, Leo set off on a journey across forests, rivers, and deserts. Along the way, he helped strangers, solved riddles, and resisted temptations. In the end, Leo didn’t find gold or jewels — the real treasure was the friendships he built and the wisdom he gained.\n\nWhen he returned home, the village celebrated him not for bringing riches, but for bringing back stories, hope, and courage they had long forgotten.\n\nAnd so, Leo’s name lived on, not in legend for wealth, but for kindness"
prediction = predict_ai_generated(example_text)
print(f"Prediction for the example text: {prediction}") # Output: 0 or 1


Prediction for the example text: 1


In [ ]:
# prompt: transform the model to my drive

# Save the model to your Google Drive
from google.colab import drive
drive.mount('/content/drive')

model_path = "/content/drive/MyDrive/my_ai_detection_model"  # Choose a path in your Google Drive

trainer.save_model(model_path)


Mounted at /content/drive
